In [ ]:
import emission.storage.timeseries.aggregate_timeseries as estag
import emission.storage.timeseries.timequery as estt
import arrow
import pandas as pd
import emission.core.get_database as edb
from emission.core.wrapper.user import User
import emission.storage.timeseries.abstract_timeseries as esta
from datetime import timedelta, date, tzinfo, datetime
import matplotlib.pyplot as plt
import matplotlib
import math
import pytz
from scipy import stats
from collections import Counter
import numpy as np
class UTC(tzinfo):
    def utcoffset(self, dt):
         return timedelta(0)
    def tzname(self, dt):
        return "UTC"
    def dst(self, dt):
        return timedelta(0)

In [ ]:
agts = estag.AggregateTimeSeries()
sep_dec_tq_data_ts = estt.TimeQuery("data.ts", arrow.get('2018-03-28', 'YYYY-MM-DD').timestamp, arrow.get('2018-06-06', 'YYYY-MM-DD').timestamp)
client_nav_events_df = agts.get_data_df("stats/client_nav_event", time_query=sep_dec_tq_data_ts)

In [ ]:
app_launched_events = client_nav_events_df[client_nav_events_df.name == "opened_app"]
information_hm = {}
emotion_hm = {}
control_hm = {}
no_clients = set()
for i in range(len(app_launched_events)):
    uuid = app_launched_events[i]['user_id']
    timestamp = app_launched_events[i]['ts']
    pt = datetime.fromtimestamp(float(timestamp))
    try:
        client = edb.get_profile_db().find_one({"user_id": uuid})['client']
        if client == 'urap-2017-information':
            if uuid in information_hm:
                information_hm[uuid].append(pt)
            else:
                information_hm[uuid] = [pt]
        elif client == 'urap-2017-emotion':
            if uuid in emotion_hm:
                emotion_hm[uuid].append(pt)
            else:
                emotion_hm[uuid] = [pt]
        elif client == 'urap-2017-control':
            if uuid in control_hm:
                control_hm[uuid].append(pt)
            else:
                control_hm[uuid] = [pt]
    except:
        no_clients.add(str(uuid))
for elem in no_clients:
    print(elem + " doesnt have a client for some reason")

In [ ]:
information_data = {}
for u in information_hm:
    earliest = datetime(2020, 12, 2, 9, 30)
    latest = datetime(2009, 12, 2, 9, 30)
    for ts in information_hm[u]:
        if ts > latest:
            latest = ts
        if ts < earliest:
            earliest = ts
    information_data[u] = (earliest, latest)

emotion_data = {}
for u in emotion_hm:
    earliest = datetime(2020, 12, 2, 9, 30)
    latest = datetime(2009, 12, 2, 9, 30)
    for ts in emotion_hm[u]:
        if ts > latest:
            latest = ts
        if ts < earliest:
            earliest = ts
    emotion_data[u] = (earliest, latest)

control_data = {}
for u in control_hm:
    earliest = datetime(2020, 12, 2, 9, 30)
    latest = datetime(2009, 12, 2, 9, 30)
    for ts in control_hm[u]:
        if ts > latest:
            latest = ts
        if ts < earliest:
            earliest = ts
    control_data[u] = (earliest, latest)

In [ ]:
def calc_weeks(d1, d2):
    monday1 = (d1 - timedelta(days=d1.weekday()))
    monday2 = (d2 - timedelta(days=d2.weekday()))
    return int(math.floor((monday2 - monday1).days / 7))

In [ ]:
mode_mapping = {0: "Unknown Type", 1: "Walk-Bike", 2: "Walk-Bike", 3: "Bus", 4: "Train", 5: "Car", 6: "Air"}
# Create a dataframe with columns user_id, carbon intensity, day, week number, and group (info/emotion).
df = pd.DataFrame()
    
for i in range(len(users)):
    user_id = users[i]
    
    # Determine group for the user.
    group = "none"
    try:
        client = edb.get_profile_db().find_one({"user_id": user_id})['client']
        if client == 'urap-2017-information':
            group = "information"
        elif client == 'urap-2017-emotion':
            group = "emotion"
        elif client == 'urap-2017-control':
            group = "control"
    except:
        continue
    
    start = arrow.get('2018-03-28', 'YYYY-MM-DD')
    end = arrow.get('2018-06-06', 'YYYY-MM-DD')
    week_val = -1
    for week in arrow.Arrow.range('week', start, end):
        ts = esta.TimeSeries.get_time_series(user_id)
        begin_ts = week.timestamp
        end_ts = (week + timedelta(weeks=1)).timestamp
        last_period_tq = estt.TimeQuery("data.start_ts", begin_ts, end_ts)
        cs_df = ts.get_data_df("analysis/inferred_section", time_query=last_period_tq)
        modes = []
        try:
            for trip in cs_df[["sensed_mode"]].iterrows():
                modes.append(mode_mapping[trip[1][0]])
        except:
            continue
        count = Counter(modes)
        proportions = {"Unknown Type": 0, "Walk-Bike": 0, "Bus": 0, "Train": 0, "Car": 0, "Air": 0}
        for i in count.most_common():
            proportions[i[0]] = i[1]/len(modes)            
        week_val += 1
        df = df.append({'uuid': user_id, 'Walking-Biking': proportions["Walk-Bike"],
                        "Bus": proportions["Bus"], "Train": proportions["Train"], "Car": proportions["Car"],
                        "Air": proportions["Air"], "Unknown Type": proportions["Unknown Type"],
                        'week': week_val, 'group': group}, ignore_index=True)
df.head()

In [ ]:
df = df.groupby(['group', 'week']).mean().drop("Unknown Type", 1).drop("Air", 1)

In [ ]:
df.head()

In [ ]:
df.plot(kind='bar', figsize=(15, 15))
plt.show()

In [ ]:
num_weeks = int(df.shape[0]/3)
weeks = np.arange(num_weeks)
print(num_weeks)

In [ ]:
weekly_installs = []
for i in range(num_weeks):
    weekly_installs.append(0)
# Create a dataframe with columns user_id, carbon intensity, day, week number, and group (info/emotion).    
for i in range(len(users)):
    user_id = users[i]
    
    # Determine group for the user.
    group = "none"
    try:
        client = edb.get_profile_db().find_one({"user_id": user_id})['client']
        if client == 'urap-2017-information':
            user = information_data[user_id]
        elif client == 'urap-2017-emotion':
            group = "emotion"
            user = emotion_data[user_id]
        elif client == 'urap-2017-control':
            group = "control"
            user = control_data[user_id]
    except:
        continue
    diff = user[0].replace(tzinfo=pytz.UTC) - arrow.get(1522195200).replace(tzinfo=pytz.UTC)
    weekly_installs[max(int(diff.days / 7), 0)] += 1
    diff = user[1].replace(tzinfo=pytz.UTC) - arrow.get(1522195200).replace(tzinfo=pytz.UTC)
    weekly_installs[max(int(diff.days / 7), 0)] -= 1
print(weekly_installs)

In [ ]:
font = {'family' : 'serif',
        'weight' : 'light',
        'size'   : 16}

matplotlib.rc('font', **font)

bike = df['Walking-Biking']
control = []
emotion = []
information = []
counter = 0
for i in bike:
    if (counter < num_weeks):
        control.append(i)
    elif counter < num_weeks * 2:
        emotion.append(i)
    else:
        information.append(i)
    counter += 1
width = np.min(np.diff(weeks))/3 - 0.08
fig, ax = plt.subplots()
fig.set_size_inches(8, 8)
size = 0.25
rects1 = ax.bar(weeks, control, size)
rects2 = ax.bar(weeks + width, emotion, size)
rects3 = ax.bar(weeks + 2*width, information, size)
ax.set_ylabel('Proportion of Trips by Foot/Bicycle')
ax.set_title('Change in Proportion of Trips for Walking/Biking')
ax.set_xlabel('Week')
ax.legend((rects1[0], rects2[0], rects3[0]), ('Control', 'Emotion', 'Information'))
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.001*height,
               '%.2f' % height,
               ha='center', va='bottom')
plt.show()

In [ ]:
bike = df['Car']
control = []
emotion = []
information = []
counter = 0
for i in bike:
    if (counter < num_weeks):
        control.append(i)
    elif counter < num_weeks * 2:
        emotion.append(i)
    else:
        information.append(i)
    counter += 1
width = np.min(np.diff(weeks))/3 - 0.08
fig, ax = plt.subplots()
fig.set_size_inches(8, 8)
size = 0.25
rects1 = ax.bar(weeks, control, size)
rects2 = ax.bar(weeks + width, emotion, size)
rects3 = ax.bar(weeks + 2*width, information, size)
ax.set_ylabel('Proportion of Trips by Car')
ax.set_title('Change in Proportion of Trips in Car')
ax.set_xlabel('Week')
ax.legend((rects1[0], rects2[0], rects3[0]), ('Control', 'Emotion', 'Information'))
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.001*height,
               '%.2f' % height,
               ha='center', va='bottom')
plt.show()

In [ ]:
bike = df['Bus']
control = []
emotion = []
information = []
counter = 0
for i in bike:
    if (counter < num_weeks):
        control.append(i)
    elif counter < num_weeks * 2:
        emotion.append(i)
    else:
        information.append(i)
    counter += 1
width = np.min(np.diff(weeks))/3 - 0.08
fig, ax = plt.subplots()
fig.set_size_inches(8, 8)
size = 0.25
rects1 = ax.bar(weeks, control, size)
rects2 = ax.bar(weeks + width, emotion, size)
rects3 = ax.bar(weeks + 2*width, information, size)
ax.set_ylabel('Proportion of Trips by Bus')
ax.set_title('Change in Proportion of Trips on Bus')
ax.set_xlabel('Week')
ax.legend((rects1[0], rects2[0], rects3[0]), ('Control', 'Emotion', 'Information'))
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.001*height,
               '%.2f' % height,
               ha='center', va='bottom')
plt.show()

## Permutation Tests

In [ ]:
mode_mapping = {0: "Unknown Type", 1: "Walk-Bike", 2: "Walk-Bike", 3: "Bus", 4: "Train", 5: "Car", 6: "Air"}
# Create a dataframe with columns user_id, carbon intensity, day, week number, and group (info/emotion).
df_totals = pd.DataFrame()
df_weeks = pd.DataFrame()
for i in range(len(users)):
    user_id = users[i]
    
    # Determine group for the user.
    group = "none"
    try:
        client = edb.get_profile_db().find_one({"user_id": user_id})['client']
        if client == 'urap-2017-information':
            group = "information"
        elif client == 'urap-2017-emotion':
            group = "emotion"
        elif client == 'urap-2017-control':
            group = "control"
    except:
        continue
    
    start = arrow.get('2018-03-28', 'YYYY-MM-DD')
    end = arrow.get('2018-06-06', 'YYYY-MM-DD')
    counts = {"Unknown Type": 0, "Walk-Bike": 0, "Bus": 0, "Train": 0, "Car": 0, "Air": 0}
    modes = []
    week_val = -1
    for week in arrow.Arrow.range('week', start, end):
        ts = esta.TimeSeries.get_time_series(user_id)
        week_modes = []
        week_counts = {"Unknown Type": 0, "Walk-Bike": 0, "Bus": 0, "Train": 0, "Car": 0, "Air": 0}
        begin_ts = week.timestamp
        end_ts = (week + timedelta(weeks=1)).timestamp
        last_period_tq = estt.TimeQuery("data.start_ts", begin_ts, end_ts)
        cs_df = ts.get_data_df("analysis/inferred_section", time_query=last_period_tq)
        week_val += 1
        try:
            for trip in cs_df[["sensed_mode"]].iterrows():
                modes.append(mode_mapping[trip[1][0]])
                week_modes.append(mode_mapping[trip[1][0]])
        except:
            continue
        count = Counter(week_modes)
        for i in count.most_common():
            week_counts[i[0]] = i[1]
        if len(week_modes) != 0:
            df_weeks = df_weeks.append({'uuid': user_id, 'WalkBike': week_counts["Walk-Bike"]/len(week_modes),
                        "Bus": week_counts["Bus"]/len(week_modes), 
                        "Car": week_counts["Car"]/len(week_modes), 'week': week_val, 'group': group}, ignore_index=True)
    count = Counter(modes)
    for i in count.most_common():
        counts[i[0]] = i[1]
    if len(modes) !=  0:
        df_totals = df_totals.append({'uuid': user_id, 'WalkBike': counts["Walk-Bike"]/len(modes),
                        "Bus": counts["Bus"]/len(modes), 
                        "Car": counts["Car"]/len(modes), 'group': group}, ignore_index=True)
df_totals.head()

In [ ]:
df_weeks.head()

In [ ]:
df_weeks.groupby(['group' , 'uuid', 'week']).sum()

In [ ]:
df_car = pd.DataFrame()
df_bus = pd.DataFrame()
df_wb = pd.DataFrame()
curr_uuid = None
for index, row  in df_weeks.groupby(['group' , 'uuid', 'week']).sum().iterrows():
    curr_car = row['Car']
    curr_bus = row['Bus']
    curr_wb = row['WalkBike']
    group = index[0]
    uuid = index[1]
    week = index[2]
    if curr_uuid == None:
        curr_uuid = uuid
    if uuid == curr_uuid:
        if week == 0:
            val = math.nan
        else:
            car_val = 100 * (curr_car - prev_car)/df_totals[df_totals.uuid == curr_uuid].iloc[0].Car
            bus_val = 100 * (curr_bus - prev_bus)/df_totals[df_totals.uuid == curr_uuid].iloc[0].Bus
            wb_val = 100 * (curr_wb - prev_wb)/df_totals[df_totals.uuid == curr_uuid].iloc[0].WalkBike
            df_car = df_car.append({'uuid': uuid, 'Proportion Change (%)': car_val, 'week': week, 'group': group}, ignore_index=True)
            df_bus = df_bus.append({'uuid': uuid, 'Proportion Change (%)': bus_val, 'week': week, 'group': group}, ignore_index=True)
            df_wb = df_wb.append({'uuid': uuid, 'Proportion Change (%)': wb_val, 'week': week, 'group': group}, ignore_index=True)
    elif uuid != curr_uuid:
        curr_uuid = uuid
    prev_car = curr_car
    prev_bus = curr_bus
    prev_wb = curr_wb
df_car_mean = df_car.dropna().groupby(['group', 'uuid']).sum()
df_bus_mean = df_bus.dropna().groupby(['group', 'uuid']).sum()
df_wb_mean = df_wb.dropna().groupby(['group', 'uuid']).sum()

In [ ]:
df_car_mean

In [ ]:
df_bus_mean

In [ ]:
df_wb_mean

In [ ]:
information_walk_bike_df = df_wb[df_wb.group == "information"]
emotion_walk_bike_df = df_wb[df_wb.group == "emotion"]
control_walk_bike_df = df_wb[df_wb.group == "control"]

information_car_df = df_car[df_car.group == "information"]
emotion_car_df = df_car[df_car.group == "emotion"]
control_car_df = df_car[df_car.group == "control"]

information_bus_df = df_bus[df_bus.group == "information"]
emotion_bus_df = df_bus[df_bus.group == "emotion"]
control_bus_df = df_bus[df_bus.group == "control"]


information_walk_bike_avg = information_walk_bike_df['Proportion Change (%)'].mean()
emotion_walk_bike_avg = emotion_walk_bike_df['Proportion Change (%)'].mean()
control_walk_bike_avg = control_walk_bike_df['Proportion Change (%)'].mean()

information_car_avg = information_car_df['Proportion Change (%)'].mean()
emotion_car_avg = emotion_car_df['Proportion Change (%)'].mean()
control_car_avg = control_car_df['Proportion Change (%)'].mean()

information_bus_avg = information_bus_df['Proportion Change (%)'].mean()
emotion_bus_avg = emotion_bus_df['Proportion Change (%)'].mean()
control_bus_avg = control_bus_df['Proportion Change (%)'].mean()

In [ ]:
ec_wb = df_wb[df_wb.group != "information"]
ic_wb = df_wb[df_wb.group != "emotion"]
ie_wb = df_wb[df_wb.group != "control"]

ec_car = df_car[df_car.group != "information"]
ic_car = df_car[df_car.group != "emotion"]
ie_car = df_car[df_car.group != "control"]

ec_bus = df_bus[df_bus.group != "information"]
ic_bus = df_bus[df_bus.group != "emotion"]
ie_bus = df_bus[df_bus.group != "control"]

### Permutation Tests

In [ ]:
def perm_test(labels, response_vars, stat_func, n):
    '''Labels: Series with two labels, Response_vars series in same order as labels
    stat_func is a function that takes in two series and returns a statistic, n is permutation numnber'''
    unique_label_counts = labels.value_counts()
    label_0 = unique_label_counts.index[0]
    label_1 = unique_label_counts.index[1]
    label_0_count = unique_label_counts[0]
    label_1_count = unique_label_counts[1]
    vals_0 = response_vars[labels == label_0]
    vals_1 = response_vars[labels == label_1]
    observed_stat = stat_func(vals_0, vals_1)
    sample_stats = np.array([])
    ind = labels
    for i in range(n):
        sampler = np.random.permutation(label_0_count + label_1_count)
        new_vals = response_vars.take(sampler).values
        df = pd.DataFrame({'vals': new_vals}, index=ind)
        vals_0 = df[df.index == label_0]['vals']
        vals_1 = df[df.index == label_1]['vals']
        stat = stat_func(vals_0, vals_1)
        sample_stats = np.append(sample_stats, stat)
    perm_mean = np.mean(sample_stats)
    plt.hist(sample_stats)
    plt.show()
    if observed_stat > perm_mean:
        return np.sum(sample_stats > observed_stat) / len(sample_stats)
    return np.sum(sample_stats < observed_stat) / len(sample_stats)

In [ ]:
def mean_diff(vals_0, vals_1):
    return np.mean(vals_0) - np.mean(vals_1)

In [ ]:
#Info vs control Walking
perm_test(ic_wb['group'], ic_wb['Proportion Change (%)'], mean_diff, 100000)

In [ ]:
#Emotion vs control Walking
perm_test(ec_wb['group'], ec_wb['Proportion Change (%)'], mean_diff, 100000)

In [ ]:
#Info vs Emotion Walking
perm_test(ie_wb['group'], ie_wb['Proportion Change (%)'], mean_diff, 100000)

In [ ]:
#Info vs Control Car
perm_test(ie_car['group'], ie_car['Proportion Change (%)'], mean_diff, 100000)

In [ ]:
#Emotion vs Control Car
perm_test(ec_car['group'], ec_car['Proportion Change (%)'], mean_diff, 100000)

In [ ]:
#Emotion vs Info Car
perm_test(ie_car['group'], ie_car['Proportion Change (%)'], mean_diff, 100000)

In [ ]:
#Info vs Control Bus
perm_test(ic_bus['group'], ic_bus['Proportion Change (%)'], mean_diff, 100000)

In [ ]:
#Emotion vs Control Bus
perm_test(ec_bus['group'], ec_bus['Proportion Change (%)'], mean_diff, 100000)

In [ ]:
#Info vs Emotion Bus
perm_test(ie_bus['group'], ie_bus['Proportion Change (%)'], mean_diff, 100000)